### 6-7 使用spark-scala调用tensorflow2.0训练好的模型
本篇文章介绍在spark中调用训练好的tensorflow模型进行预测的方法。

本文内容的学习需要一定的spark和scala基础。

如果使用pyspark的话会比较简单，只需要在每个excutor上用Python加载模型分别预测就可以了。

但工程上为了性能考虑，通常使用的是scala版本的spark。

本篇文章我们通过TensorFlow for Java 在spark中调用训练好的tensorflow模型。

利用spark的分布式计算能力，从而可以让训练好的tensorflow模型在成百上千的机器上分布式并行执行模型推断。

### 〇，spark-scala调用tensorflow模型概述
在spark(scala)中调用tensorflow模型进行预测需要完成以下几个步骤。

（1）准备protobuf模型文件

（2）创建spark(scala)项目，在项目中添加java版本的tensorflow对应的jar包依赖

（3）在spark(scala)项目中driver端加载tensorflow模型调试成功

（4）在spark(scala)项目中通过RDD在excutor上加载tensorflow模型调试成功

（5） 在spark(scala)项目中通过DataFrame在excutor上加载tensorflow模型调试成功

### 一，准备protobuf模型文件
我们使用tf.keras 训练一个简单的线性回归模型，并保存成protobuf文件。

In [2]:
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers

## 样本数量
n = 800

## 生成测试用数据集
X = tf.random.uniform([n,2],minval=-10,maxval=10) 
w0 = tf.constant([[2.0],[-1.0]])
b0 = tf.constant(3.0)

Y = X@w0 + b0 + tf.random.normal([n,1],mean = 0.0,stddev= 2.0)  # @表示矩阵乘法,增加正态扰动

## 建立模型
tf.keras.backend.clear_session()
inputs = layers.Input(shape = (2,),name ="inputs") #设置输入名字为inputs
outputs = layers.Dense(1, name = "outputs")(inputs) #设置输出名字为outputs
linear = models.Model(inputs = inputs,outputs = outputs)
linear.summary()

## 使用fit方法进行训练
linear.compile(optimizer="rmsprop",loss="mse",metrics=["mae"])
linear.fit(X,Y,batch_size = 8,epochs = 100)  

tf.print("w = ",linear.layers[1].kernel)
tf.print("b = ",linear.layers[1].bias)

## 将模型保存成pb格式文件
export_path = "../data/linear_model/"
version = "1"       #后续可以通过版本号进行模型版本迭代与管理
linear.save(export_path+version, save_format="tf")


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 2)]               0         
_________________________________________________________________
outputs (Dense)              (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________
Train on 800 samples
Epoch 1/100
800/800 [==============================] - 1s 1ms/sample - loss: 194.2585 - mae: 11.9687
Epoch 2/100
800/800 [==============================] - 0s 178us/sample - loss: 179.9152 - mae: 11.5112
Epoch 3/100
800/800 [==============================] - 0s 256us/sample - loss: 166.0658 - mae: 11.0498
Epoch 4/100
800/800 [==============================] - 0s 391us/sample - loss: 152.3754 - mae: 10.5779
Epoch 5/100
800/800 [==============================] - 0s 372us/sample - loss: 139.4555 - mae: 10.

800/800 [==============================] - 0s 313us/sample - loss: 3.8221 - mae: 1.5608
Epoch 76/100
800/800 [==============================] - 0s 182us/sample - loss: 3.8210 - mae: 1.5617
Epoch 77/100
800/800 [==============================] - 0s 215us/sample - loss: 3.8222 - mae: 1.5604
Epoch 78/100
800/800 [==============================] - 0s 192us/sample - loss: 3.8221 - mae: 1.5613
Epoch 79/100
800/800 [==============================] - 0s 184us/sample - loss: 3.8220 - mae: 1.5611
Epoch 80/100
800/800 [==============================] - 0s 214us/sample - loss: 3.8217 - mae: 1.5608
Epoch 81/100
800/800 [==============================] - 0s 212us/sample - loss: 3.8216 - mae: 1.5614
Epoch 82/100
800/800 [==============================] - 0s 187us/sample - loss: 3.8212 - mae: 1.5614
Epoch 83/100
800/800 [==============================] - 0s 213us/sample - loss: 3.8219 - mae: 1.5609
Epoch 84/100
800/800 [==============================] - 0s 169us/sample - loss: 3.8215 - mae: 1.5607
Epo

W0423 00:08:28.954216 4365151680 deprecation.py:506] From /Users/ting/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
!ls {export_path+version}

assets	saved_model.pb	variables


In [4]:
# 查看模型文件相关信息
!saved_model_cli show --dir {export_path+str(version)} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 2)
        name: serving_default_inputs:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['outputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0423 00:09:25.968805 4616895936 deprecation.py:506] From /Users/ting/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/ops/r

### 二，创建spark(scala)项目，在项目中添加java版本的tensorflow对应的jar包依赖

#### 如果使用maven管理项目，需要添加如下 jar包依赖

In [5]:
<!-- https://mvnrepository.com/artifact/org.tensorflow/tensorflow -->
<dependency>
    <groupId>org.tensorflow</groupId>
    <artifactId>tensorflow</artifactId>
    <version>1.15.0</version>
</dependency>

SyntaxError: invalid syntax (<ipython-input-5-4a6e734c6a2b>, line 1)

也可以从下面网址中直接下载 org.tensorflow.tensorflow的jar包

以及其依赖的org.tensorflow.libtensorflow 和 org.tensorflowlibtensorflow_jni的jar包 放到项目中。

https://mvnrepository.com/artifact/org.tensorflow/tensorflow/1.15.0

### 三， 在spark(scala)项目中driver端加载tensorflow模型调试成功
我们的示范代码在jupyter notebook中进行演示，需要安装toree以支持spark(scala)。

In [6]:
import scala.collection.mutable.WrappedArray
import org.{tensorflow=>tf}

//注：load函数的第二个参数一般都是“serve”，可以从模型文件相关信息中找到

val bundle = tf.SavedModelBundle 
   .load("/Users/liangyun/CodeFiles/eat_tensorflow2_in_30_days/data/linear_model/1","serve")

//注：在java版本的tensorflow中还是类似tensorflow1.0中静态计算图的模式，需要建立Session, 指定feed的数据和fetch的结果, 然后 run.
//注：如果有多个数据需要喂入，可以连续用用多个feed方法
//注：输入必须是float类型

val sess = bundle.session()
val x = tf.Tensor.create(Array(Array(1.0f,2.0f),Array(2.0f,3.0f)))
val y =  sess.runner().feed("serving_default_inputs:0", x)
         .fetch("StatefulPartitionedCall:0").run().get(0)

val result = Array.ofDim[Float](y.shape()(0).toInt,y.shape()(1).toInt)
y.copyTo(result)

if(x != null) x.close()
if(y != null) y.close()
if(sess != null) sess.close()
if(bundle != null) bundle.close()  

result

SyntaxError: invalid syntax (<ipython-input-6-765bce07b38f>, line 2)

### 四，在spark(scala)项目中通过RDD在excutor上加载tensorflow模型调试成功
下面我们通过广播机制将Driver端加载的TensorFlow模型传递到各个excutor上，并在excutor上分布式地调用模型进行推断。